# TWFE Difference-in-Differences Regression

## Objective

This notebook estimates causal lift using Two-Way Fixed Effects (TWFE) Difference-in-Differences regression on the purchase-based panel.

**What TWFE estimates:**
TWFE regression pools across campaigns to estimate an average treatment effect (ATE) while controlling for:
- Household-level fixed effects (time-invariant household characteristics)
- Week-level fixed effects (aggregate time-varying confounds)

**Why regression instead of per-campaign 2x2 means:**
With 20 campaigns, many have incomplete 2x2 structures (missing treated-control or pre-post cells). TWFE pools data across campaigns and time periods to estimate a pooled ATE even when campaign-level 2x2 is sparse.

**Inputs:**
- `data/intermediate/did_campaign_panel_purchase.parquet`
- `did_campaign_estimates_purchase.parquet`, `did_campaign_estimates_purchase_valid.parquet`, `did_campaign_estimates_purchase_invalid.parquet`, `did_overall_estimate_purchase.parquet`

**Outputs:**
- Summary tables (JSON, CSV)
- Model coefficients (CSV)
- Diagnostics report (TXT)
- Figures (PNG) in `results/modeling/figures/`


## Environment & Imports


In [26]:
import os
import sys
import json
import logging
from pathlib import Path
from datetime import datetime, timezone
from typing import Dict, List

import pandas as pd
import numpy as np
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print(f"Python executable: {sys.executable}")


Python executable: /Users/rajnishpanwar/.venv/bin/python


## Configuration


In [27]:
def find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(12):
        if (p / "README.md").exists():
            return p
        p = p.parent
    raise RuntimeError("Repository root not found (README.md missing)")


REPO_ROOT = find_repo_root(Path.cwd())
DATA_DIR = REPO_ROOT / "data" / "intermediate"
OUT_DIR = REPO_ROOT / "results" / "modeling"
FIG_DIR = OUT_DIR / "figures"
RUN_MANIFESTS_DIR = REPO_ROOT / "results" / "run_manifests"

OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)
RUN_MANIFESTS_DIR.mkdir(parents=True, exist_ok=True)

logger.info(f"Repository root: {REPO_ROOT}")


2026-01-14 18:01:33,785 - INFO - Repository root: /Users/rajnishpanwar/Desktop/Casual Analytics


## Load Data


In [28]:
panel_file = DATA_DIR / "did_campaign_panel_purchase.parquet"
if not panel_file.exists():
    raise FileNotFoundError(f"Panel file not found: {panel_file}")

df_panel = pd.read_parquet(panel_file)
logger.info(f"Loaded panel: {len(df_panel)} rows, {len(df_panel.columns)} columns")


2026-01-14 18:01:33,821 - INFO - Loaded panel: 122536 rows, 11 columns


## Data Quality Gates


In [29]:
required_cols = ["campaign_id", "household_id", "week_number", "treated", "post", "total_sales_value", "total_units"]
missing_cols = [col for col in required_cols if col not in df_panel.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")

df_panel["treated"] = df_panel["treated"].astype(int)
df_panel["post"] = df_panel["post"].astype(int)

df_panel["did"] = df_panel["treated"] * df_panel["post"]

df_panel["y_sales"] = pd.to_numeric(df_panel["total_sales_value"], errors="coerce")
df_panel["y_units"] = pd.to_numeric(df_panel["total_units"], errors="coerce")
df_panel["y_log_sales"] = np.log1p(np.maximum(df_panel["total_sales_value"], 0))

treated_post_crosstab = pd.crosstab(df_panel["treated"], df_panel["post"], margins=True)
print("2x2 Structure (treated x post, rows):")
print(treated_post_crosstab)

n_rows = len(df_panel)
n_households = df_panel["household_id"].nunique()
n_campaigns = df_panel["campaign_id"].nunique()

logger.info(f"Summary: {n_rows} rows, {n_households} households, {n_campaigns} campaigns")


2026-01-14 18:01:33,854 - INFO - Summary: 122536 rows, 30 households, 1584 campaigns


2x2 Structure (treated x post, rows):
post         0      1     All
treated                      
0        57256  64413  121669
1          408    459     867
All      57664  64872  122536


## Feature Engineering

Sales values can be negative due to discounts/returns. For log transformation, we shift by the minimum value so all values are non-negative before applying log1p.


In [30]:
def run_twfe(df: pd.DataFrame, outcome_col: str, cluster_col: str = "household_id"):
    y = df[outcome_col].values
    
    X_did = pd.DataFrame({"did": df["did"].astype(float)})
    X_hh = pd.get_dummies(df["household_id"], prefix="hh", drop_first=True, dtype=float)
    X_week = pd.get_dummies(df["week_number"], prefix="week", drop_first=True, dtype=float)
    
    X = pd.concat([X_did, X_hh, X_week], axis=1)
    X = add_constant(X, prepend=False)
    X = X.astype(float)
    
    param_names = X.columns.tolist()
    
    model = OLS(y, X)
    result = model.fit(cov_type="cluster", cov_kwds={"groups": df[cluster_col]})
    
    result.param_names = param_names
    
    return result


## Modeling Plan

Fit TWFE DiD models with household and week fixed effects:
- Model A: y_sales ~ did + C(household_id) + C(week_number)
- Model B: y_log_sales ~ did + C(household_id) + C(week_number)
- Model C: y_units ~ did + C(household_id) + C(week_number)

Standard errors: clustered at household_id level. Also compute robust HC1 standard errors for comparison.


In [31]:
models: Dict[str, object] = {}
outcomes = ["y_sales", "y_log_sales", "y_units"]

for outcome in outcomes:
    logger.info(f"Fitting model for {outcome}")
    result = run_twfe(df_panel, outcome)
    models[outcome] = result
    logger.info(f"  R²: {result.rsquared:.4f}, N: {result.nobs}")

logger.info("All models fitted")


2026-01-14 18:01:33,871 - INFO - Fitting model for y_sales
2026-01-14 18:01:34,811 - INFO -   R²: 0.3296, N: 122536.0
2026-01-14 18:01:34,812 - INFO - Fitting model for y_log_sales
2026-01-14 18:01:35,881 - INFO -   R²: 0.3717, N: 122536.0
2026-01-14 18:01:35,881 - INFO - Fitting model for y_units
2026-01-14 18:01:36,787 - INFO -   R²: 0.2433, N: 122536.0
2026-01-14 18:01:36,788 - INFO - All models fitted


## Save Outputs


In [32]:
results_summary = []
for outcome in outcomes:
    result = models[outcome]
    
    if hasattr(result, 'param_names') and "did" in result.param_names:
        did_idx = result.param_names.index("did")
        did_coef = result.params.iloc[did_idx]
        did_se = result.bse.iloc[did_idx]
        did_pval = result.pvalues.iloc[did_idx]
        ci = result.conf_int().iloc[did_idx]
        ci_low, ci_high = ci[0], ci[1]
    elif hasattr(result.params, 'index') and "did" in result.params.index:
        did_coef = result.params["did"]
        did_se = result.bse["did"]
        did_pval = result.pvalues["did"]
        ci = result.conf_int().loc["did"]
        ci_low, ci_high = ci[0], ci[1]
    else:
        did_coef = 0
        did_se = np.nan
        did_pval = 1.0
        ci_low, ci_high = np.nan, np.nan
    
    results_summary.append({
        "outcome": outcome.replace("y_", ""),
        "coef": float(did_coef),
        "se_clustered": float(did_se),
        "pvalue": float(did_pval),
        "ci_low": float(ci_low),
        "ci_high": float(ci_high),
        "nobs": int(result.nobs),
        "n_households": n_households
    })

df_results = pd.DataFrame(results_summary)
df_results.to_csv(OUT_DIR / "twfe_did_summary.csv", index=False)

result_sales = models["y_sales"]
if hasattr(result_sales, 'param_names'):
    df_coefs = pd.DataFrame({"param": result_sales.param_names, "coefficient": result_sales.params.values})
elif hasattr(result_sales.params, 'index'):
    df_coefs = pd.DataFrame({"param": result_sales.params.index, "coefficient": result_sales.params.values})
else:
    df_coefs = pd.DataFrame({"param": range(len(result_sales.params)), "coefficient": result_sales.params})
df_coefs.to_csv(OUT_DIR / "twfe_coefficients.csv", index=False)

logger.info(f"Outputs saved to {OUT_DIR}")


/Users/rajnishpanwar/.venv/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/Users/rajnishpanwar/.venv/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/Users/rajnishpanwar/.venv/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
2026-01-14 18:01:36,809 - INFO - Outputs saved to /Users/rajnishpanwar/Desktop/Casual Analytics/results/modeling


## Run Manifest


In [33]:
def get_package_versions() -> Dict[str, str]:
    versions = {}
    for pkg in ["pandas", "numpy", "statsmodels"]:
        try:
            mod = __import__(pkg)
            versions[pkg] = getattr(mod, "__version__", "unknown")
        except ImportError:
            versions[pkg] = "not_installed"
    return versions


manifest = {
    "timestamp_utc": datetime.now(timezone.utc).isoformat(),
    "input_parquet": str(panel_file),
    "output_files": [
        str(OUT_DIR / "twfe_did_summary.csv"),
        str(OUT_DIR / "twfe_coefficients.csv")
    ],
    "n_rows": n_rows,
    "n_households": n_households,
    "n_campaigns": n_campaigns,
    "package_versions": get_package_versions(),
    "python_executable": sys.executable
}

manifest_path = RUN_MANIFESTS_DIR / f"twfe_manifest_{datetime.now(timezone.utc).strftime('%Y%m%d_%H%M%S')}.json"
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

logger.info(f"Manifest saved to {manifest_path}")


2026-01-14 18:01:36,815 - INFO - Manifest saved to /Users/rajnishpanwar/Desktop/Casual Analytics/results/run_manifests/twfe_manifest_20260114_180136.json


## Next Steps

- **Notebook 03**: Event study regression (leads/lags) to test parallel trends assumption
- **Notebook 04**: Uplift modeling (T-learner / X-learner style) for heterogeneous treatment effects
